In [1]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Conv3D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import CSVLogger
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import read_csv
import math
import seaborn as sns
from collections import deque
import copy

In [2]:
def file_generator(data_path,data_files,temporal_stride=1,temporal_length=8):
    print(data_files)
    for f in data_files: 
        tmp_df = pd.read_csv(os.path.join(data_path,f), sep=',')
        label_list = list(tmp_df['label'])  
        total_images = len(label_list) 
        if total_images==temporal_length: 
            num_samples = int((total_images-temporal_length)/temporal_stride)+1
            print ('num of frames: {}: {}'.format(f,num_samples))
            img_list = list(tmp_df['FileName'])
            #print(img_list)
        else: # if the number of frames < than temporal length , discard it
            print ('num of frames is less than temporal length; hence discarding this file-{}'.format(f))
            continue

        start_frame = 0
        samples = deque()  
        samp_count=0  
        for img in img_list:
            samples.append(img)
            if len(samples)==temporal_length: 
                samples_c=copy.deepcopy(samples) 
                samp_count+=1
                for t in range(temporal_stride): 
                    samples.popleft()
                yield samples_c,label_list[0]  

In [3]:
def load_samples(data_cat='train',temporal_stride=1,temporal_length=8):
    data_path = os.path.join('data_files',data_cat)
    data_files = os.listdir(data_path)
    #print(data_files)
    # define a generator to read the samples
    file_gen = file_generator(data_path,data_files,temporal_stride,temporal_length)
    iterator = True
    data_list = []
    while iterator:
        try:
            x,y = next(file_gen)
            x=list(x)
            data_list.append([x,y])
        except Exception as e:
            print ('Exception: ',e)
            iterator = False
            print ('end of data generator')
    return data_list

In [4]:
train_data = load_samples(data_cat='train',temporal_stride=1,temporal_length=8)

['cluster1.csv', 'cluster0.csv', 'cluster2.csv', 'cluster3.csv', 'cluster7.csv', 'cluster6.csv', 'cluster4.csv', 'cluster5.csv', 'cluster8.csv', 'cluster9.csv']
num of frames: cluster1.csv: 1
num of frames: cluster0.csv: 1
num of frames: cluster2.csv: 1
num of frames: cluster3.csv: 1
num of frames: cluster7.csv: 1
num of frames: cluster6.csv: 1
num of frames: cluster4.csv: 1
num of frames: cluster5.csv: 1
num of frames: cluster8.csv: 1
num of frames: cluster9.csv: 1
Exception:  
end of data generator


In [5]:
print ('Total number of train samples:',len(train_data))

Total number of train samples: 10


In [6]:
train_data[0]

[['/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame00.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame01.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame02.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame03.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame04.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame05.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame06.npy',
  '/Users/jieunyoo/april12_3dCNN/figures/cluster1/event000001_frame07.npy'],
 -0.0398580703991084]

In [7]:
def shuffle_data(samples):
    data = shuffle(samples,random_state=2)
    return data

In [8]:
def data_generator(data,batch_size=2,temporal_padding='same',shuffle=True):               
    num_samples = len(data)
    print(num_samples)
    if shuffle:
        data = shuffle_data(data)
    while True:   
        for offset in range(0, num_samples, batch_size):
            print ('starting index: ', offset) 
            batch_samples = data[offset:offset+batch_size]
            
            X_train = []
            y_train = []
            
            for batch_sample in batch_samples: 
                print(batch_sample)
                # Load image (X)
                x = batch_sample[0]
                y = batch_sample[1]
                temp_data_list = []
                for img in x:
                    try:
                        img = np.load(img)                        
                        temp_data_list.append(img)
                    except Exception as e:
                        print (e)
                        print ('error reading in frame: ',img)                      

                X_train.append(temp_data_list)
                y_train.append(y)

            X_train = np.array(X_train)   
            y_train = np.array(y_train)
            
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        
            print(X_train.shape)               
            yield X_train, y_train

            
print(len(train_data))
train_generator = data_generator(train_data,batch_size=2,shuffle=True)
#x,y = next(train_generator)
#print ('x shape: ',x.shape) 
#print(X_train.shape)
#xx,yy = next(train_generator)

10


In [9]:
def get_model(num_classes=1):
    model = Sequential()
    model.add(
    TimeDistributed(
        Conv2D(64, (3,3), activation='relu'), 
        input_shape=(8, 13, 21, 1))
    )
    model.add(
    TimeDistributed(
        Conv2D(64, (3,3), activation='relu')
        )
    )
    model.add(
    TimeDistributed(
        Flatten()
        )
    )        
    model.add(LSTM(1024, activation='relu', return_sequences=False))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(.01))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='Huber', optimizer=Adam(), metrics=['mean_squared_error'])
    model.summary()
    return model

model = get_model()
    

2022-04-18 08:41:30.488483: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 8, 11, 19, 64)    640       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 8, 9, 17, 64)     36928     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 8, 9792)          0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 1024)              44306432  
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                        

In [10]:
hist = model.fit(train_generator,steps_per_epoch=len(train_data),epochs=2)

10
starting index:  0
[['/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame03.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame04.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame05.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame06.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster7/event000007_frame07.npy'], 0.2371483913959306]
[['/Users/jieunyoo/april12_3dCNN/figures/cluster0/event000000_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster0/event000000_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster0/event000000_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster0/event000000_frame03.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster0/event000000_frame

 6/10 [=================>............] - ETA: 3s - loss: 1.2297 - mean_squared_error: 8.8991 starting index:  4
[['/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame03.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame04.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame05.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame06.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster5/event000005_frame07.npy'], 0.0981641229120418]
[['/Users/jieunyoo/april12_3dCNN/figures/cluster2/event000002_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster2/event000002_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster2/event000002_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster2/e

 2/10 [=====>........................] - ETA: 7s - loss: 0.0809 - mean_squared_error: 0.1618starting index:  6
[['/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame03.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame04.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame05.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame06.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster3/event000003_frame07.npy'], 0.1443460621105261]
[['/Users/jieunyoo/april12_3dCNN/figures/cluster4/event000004_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster4/event000004_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster4/event000004_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster4/ev

 8/10 [=======================>......] - ETA: 1s - loss: 0.0302 - mean_squared_error: 0.0603starting index:  8
[['/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame03.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame04.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame05.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame06.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster9/event000009_frame07.npy'], 0.175330401018919]
[['/Users/jieunyoo/april12_3dCNN/figures/cluster8/event000008_frame00.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster8/event000008_frame01.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster8/event000008_frame02.npy', '/Users/jieunyoo/april12_3dCNN/figures/cluster8/eve